## Convertendo de spacy para coletar apenas os textos e suas entidades

In [8]:
import json
import os
import time
from dotenv import load_dotenv
import google.generativeai as genai
from collections import defaultdict

In [9]:
def convert_spacy_to_gold(dataset):
    gold = []
    
    for text, ann in dataset:
        entities = []
        
        # ann é um dict: {"entities": [(start, end, label), ...]}
        for start, end, label in ann["entities"]:
            entities.append({
                "label": label,
                "text": text[start:end],
                "start": start,
                "end": end
            })
        
        gold.append({
            "text": text,
            "entities": entities
        })

    return gold


dataset = [('O presidente Luiz Inácio Lula da Silva anunciou nesta quarta-feira um novo pacote de investimentos em infraestrutura. O programa, batizado de Avança Brasil, prevê R$ 200 bilhões em obras de transporte e energia. Entre os projetos, está a duplicação da BR-153, que liga Goiás ao Tocantins. O ministro da Fazenda, Fernando Haddad, afirmou que os recursos virão em parte do Banco Nacional de Desenvolvimento Econômico e Social (BNDES). O objetivo é gerar dois milhões de empregos diretos até 2026. O governo espera atrair também capital estrangeiro para os projetos de energia solar e eólica. Durante o evento, realizado em Brasília, Lula destacou a importância da integração regional. A cerimônia contou com a presença de governadores, prefeitos e empresários. O presidente disse que a prioridade será o desenvolvimento sustentável. Especialistas consideram o programa uma das maiores apostas do governo federal desde o PAC.', {'entities': [(631, 635, 'PER'), (312, 327, 'PER'), (13, 38, 'PER'), (163, 177, 'VALOR'), (269, 274, 'LOC'), (278, 287, 'LOC'), (621, 629, 'LOC'), (2, 12, 'CARGO'), (291, 310, 'CARGO'), (203, 210, 'TEMA'), (389, 423, 'ORG'), (425, 430, 'ORG'), (761, 771, 'CARGO'), (102, 116, 'TEMA'), (190, 200, 'TEMA'), (566, 579, 'TEMA'), (802, 829, 'TEMA')]}), ('A Petrobras anunciou um novo reajuste no preço da gasolina, que passará a custar R$ 5,29 por litro nas refinarias. O aumento será de 4,2% em média, segundo comunicado divulgado nesta segunda-feira. A decisão ocorre após a alta do barril de petróleo no mercado internacional. O presidente da companhia, Jean Paul Prates, afirmou que o reajuste busca equilibrar os custos de importação. O impacto nos postos de combustível deve ser sentido já nos próximos dias. Economistas alertam para possível pressão inflacionária no curto prazo. O Ministério da Fazenda avalia medidas para conter os efeitos sobre o consumidor. Sindicatos de caminhoneiros criticaram o aumento e pedem subsídios temporários. Em nota, o governo reforçou que a política de preços segue critérios técnicos. O mercado financeiro reagiu com leve queda nas ações da estatal.', {'entities': [(2, 11, 'ORG'), (133, 137, 'VALOR'), (81, 88, 'VALOR'), (302, 318, 'PER'), (277, 287, 'CARGO'), (534, 555, 'ORG'), (252, 273, 'TEMA'), (775, 793, 'TEMA')]}), ('Pesquisadores da Universidade de São Paulo desenvolveram um novo teste rápido para detecção de dengue. O exame promete identificar o vírus em menos de 10 minutos. O projeto é coordenado pelo Instituto de Ciências Biomédicas e contou com apoio da Fundação de Amparo à Pesquisa do Estado de São Paulo (FAPESP). A nova tecnologia utiliza nanopartículas de ouro para detectar anticorpos no sangue. Segundo os cientistas, o custo do teste será de apenas R$ 15 por unidade. A expectativa é que o produto esteja disponível no SUS até o final de 2025. O Ministério da Saúde demonstrou interesse em ampliar a produção nacional. Nos últimos meses, o Brasil registrou aumento de 30% nos casos de dengue. Especialistas acreditam que a ferramenta poderá reduzir o tempo de diagnóstico e salvar vidas. O estudo foi publicado na revista científica Nature Biomedical Engineering.', {'entities': [(17, 42, 'ORG'), (191, 223, 'ORG'), (519, 522, 'ORG'), (546, 565, 'ORG'), (449, 454, 'VALOR'), (668, 671, 'VALOR'), (95, 101, 'TEMA'), (316, 326, 'TEMA'), (640, 646, 'LOC'), (685, 691, 'TEMA'), (246, 298, 'ORG'), (300, 306, 'ORG')]}), ('O Google anunciou a abertura de um novo centro de pesquisa em inteligência artificial no Brasil. A unidade será instalada na cidade de Belo Horizonte, em Minas Gerais. O investimento total é estimado em US$ 120 milhões. O objetivo é desenvolver soluções locais em linguagem natural e visão computacional. A empresa informou que pretende contratar 200 engenheiros e cientistas de dados até 2026. O projeto faz parte da estratégia global do Google de expandir seus laboratórios de IA para países emergentes. O ministro da Ciência, Tecnologia e Inovação, Luciana Santos, participou da cerimônia de inauguração. Ela destacou que o Brasil tem potencial para se tornar referência em IA ética e sustentável. Pesquisadores da UFMG e da USP serão parceiros no desenvolvimento dos algoritmos. A expectativa é que o centro estimule startups e a inovação tecnológica na região.', {'entities': [(2, 8, 'ORG'), (718, 722, 'ORG'), (728, 731, 'ORG'), (439, 445, 'ORG'), (552, 566, 'PER'), (135, 149, 'LOC'), (154, 166, 'LOC'), (508, 516, 'CARGO'), (627, 633, 'LOC'), (677, 685, 'TEMA'), (688, 699, 'TEMA'), (834, 854, 'TEMA'), (62, 85, 'TEMA'), (264, 281, 'TEMA'), (284, 303, 'TEMA'), (351, 362, 'CARGO'), (365, 384, 'CARGO'), (203, 218, 'VALOR'), (89, 95, 'LOC'), (347, 350, 'VALOR')]}), ('O Banco Central manteve a taxa Selic em 10,75% ao ano, interrompendo o ciclo de cortes iniciado em 2023. A decisão foi unânime entre os membros do Comitê de Política Monetária (Copom). Segundo o comunicado, a autoridade monetária avaliou que a inflação ainda apresenta riscos significativos. O presidente do BC, Roberto Campos Neto, destacou que o cenário externo exige cautela. Analistas do mercado financeiro esperavam uma redução de 0,25 ponto percentual. Após o anúncio, o real se valorizou frente ao dólar. Investidores interpretaram a decisão como sinal de responsabilidade fiscal. O Ministério da Fazenda afirmou que continua comprometido com o controle das contas públicas. Especialistas acreditam que a taxa pode voltar a cair em 2026, caso a inflação permaneça sob controle. O próximo encontro do Copom está marcado para dezembro.', {'entities': [(2, 15, 'ORG'), (590, 611, 'ORG'), (312, 331, 'PER'), (244, 252, 'TEMA'), (392, 410, 'TEMA'), (26, 36, 'TEMA'), (807, 812, 'ORG'), (40, 46, 'VALOR'), (752, 760, 'TEMA'), (563, 586, 'TEMA'), (147, 175, 'ORG'), (177, 182, 'ORG'), (294, 310, 'CARGO'), (436, 457, 'VALOR')]}), ('Uma onda de calor atingiu o Centro-Oeste e o Sudeste do Brasil nesta semana, com temperaturas acima de 40 graus. O Instituto Nacional de Meteorologia (Inmet) emitiu alerta de grande perigo para 12 estados. Em Goiânia, a sensação térmica chegou a 43°C nesta terça-feira. A população foi orientada a evitar exposição prolongada ao sol e a beber muita água. A Defesa Civil informou aumento de casos de desidratação e desmaios. Escolas públicas suspenderam atividades esportivas ao ar livre. Especialistas afirmam que o fenômeno está relacionado ao El Niño. Meteorologistas alertam que o calor deve persistir até o final do mês. Governos estaduais estudam medidas emergenciais para evitar colapso no sistema elétrico. O Ministério da Saúde recomendou cuidados redobrados com crianças e idosos.', {'entities': [(4, 17, 'TEMA'), (220, 236, 'TEMA'), (696, 712, 'TEMA'), (357, 369, 'ORG'), (716, 735, 'ORG'), (28, 40, 'LOC'), (45, 52, 'LOC'), (56, 62, 'LOC'), (554, 569, 'CARGO'), (652, 672, 'TEMA'), (209, 216, 'LOC'), (453, 474, 'TEMA'), (115, 149, 'ORG'), (103, 111, 'VALOR'), (151, 156, 'ORG'), (246, 250, 'VALOR'), (194, 196, 'VALOR')]}), ('A Organização das Nações Unidas (ONU) divulgou um relatório alertando para o aumento do desmatamento na Amazônia. Segundo o documento, o Brasil perdeu 12 mil km² de floresta em 2024. O número representa um crescimento de 8% em relação ao ano anterior. O relatório foi apresentado em Genebra, na Suíça. O governo brasileiro afirmou que está reforçando as operações de fiscalização. O ministro do Meio Ambiente, Marina Silva, disse que o desmatamento ilegal será combatido com mais rigor. Organizações ambientais criticaram a lentidão das medidas. O Greenpeace alertou que a destruição ameaça a biodiversidade e o equilíbrio climático. Cientistas destacam que a recuperação da floresta exige políticas de longo prazo. O relatório também aponta avanço do garimpo em áreas de proteção indígena.', {'entities': [(2, 31, 'ORG'), (548, 558, 'ORG'), (295, 300, 'LOC'), (283, 290, 'LOC'), (137, 143, 'LOC'), (104, 112, 'LOC'), (383, 391, 'CARGO'), (221, 223, 'VALOR'), (88, 100, 'TEMA'), (367, 379, 'TEMA'), (436, 455, 'TEMA'), (593, 607, 'TEMA'), (612, 632, 'TEMA'), (410, 422, 'PER'), (33, 36, 'ORG'), (151, 161, 'VALOR')]}), ('O Ministério da Educação anunciou mudanças no Exame Nacional do Ensino Médio (Enem) a partir de 2026. A principal novidade será a inclusão de questões adaptativas, que variam de acordo com o desempenho do aluno. O objetivo é tornar o exame mais dinâmico e preciso na avaliação. O ministro Camilo Santana afirmou que o novo formato seguirá modelos utilizados nos Estados Unidos e na Europa. A redação continuará sendo obrigatória. O Inep informou que os testes pilotos começarão no segundo semestre de 2025. Especialistas acreditam que o modelo pode reduzir desigualdades regionais. Professores pedem mais transparência sobre o algoritmo de correção. Estudantes demonstraram curiosidade e preocupação nas redes sociais. O MEC garante que a transição será gradual e segura.', {'entities': [(2, 24, 'ORG'), (432, 436, 'ORG'), (721, 724, 'ORG'), (362, 376, 'LOC'), (382, 388, 'LOC'), (280, 288, 'CARGO'), (582, 593, 'CARGO'), (650, 660, 'CARGO'), (289, 303, 'PER'), (557, 580, 'TEMA'), (627, 648, 'TEMA'), (704, 717, 'TEMA'), (78, 82, 'ORG'), (46, 76, 'ORG')]}), ('A startup brasileira AgroTech Solutions lançou um sistema de irrigação inteligente para pequenas propriedades rurais. A tecnologia utiliza sensores e algoritmos de aprendizado de máquina para otimizar o uso da água. O produto foi desenvolvido em parceria com a Embrapa e a Universidade Federal de Viçosa. Segundo os criadores, o sistema pode reduzir o consumo de água em até 40%. O projeto recebeu apoio do BNDES e da Finep. Agricultores do interior de Minas Gerais participam dos primeiros testes. O CEO da AgroTech, Rafael Costa, afirmou que o objetivo é democratizar o acesso à tecnologia no campo. O sistema também fornece relatórios automáticos sobre produtividade e umidade do solo. O Ministério da Agricultura destacou o potencial da iniciativa para combater os efeitos das secas. A empresa planeja exportar o produto para outros países da América Latina.', {'entities': [(21, 39, 'ORG'), (508, 516, 'ORG'), (261, 268, 'ORG'), (273, 303, 'ORG'), (418, 423, 'ORG'), (453, 465, 'LOC'), (847, 861, 'LOC'), (150, 186, 'TEMA'), (352, 367, 'TEMA'), (375, 378, 'VALOR'), (407, 412, 'ORG'), (501, 504, 'CARGO'), (518, 530, 'PER'), (691, 716, 'ORG'), (50, 70, 'TEMA'), (120, 130, 'TEMA'), (139, 147, 'TEMA'), (656, 669, 'TEMA'), (672, 687, 'TEMA'), (581, 591, 'TEMA')]}),('O Congresso Nacional aprovou nesta terça-feira o novo Marco Legal da Indústria Verde. O projeto prevê incentivos fiscais para empresas que reduzirem emissões de carbono e investirem em energia limpa. A proposta segue agora para sanção do presidente Lula. O ministro da Indústria e Comércio, Geraldo Alckmin, afirmou que a medida pode atrair até R$ 80 bilhões em investimentos privados. Ambientalistas elogiaram o avanço, mas cobram regras mais rígidas contra a poluição industrial. A Confederação Nacional da Indústria acredita que o marco impulsionará exportações de produtos sustentáveis. O texto prevê ainda linhas de crédito especiais do BNDES para empresas que cumprirem metas ambientais.', {'entities': [(2, 20, 'ORG'), (484, 518, 'ORG'), (249, 253, 'PER'), (291, 306, 'PER'), (345, 358, 'VALOR'), (257, 265, 'CARGO'), (185, 198, 'TEMA'), (461, 480, 'TEMA')]}), ('A Embraer anunciou uma parceria com a Força Aérea Brasileira para o desenvolvimento de um novo modelo de avião elétrico de treinamento militar. O projeto, chamado de E-TALON, deve ter o primeiro protótipo concluído até 2027. A aeronave utilizará baterias de lítio nacionais produzidas no Ceará. O presidente da Embraer, Francisco Gomes Neto, destacou que o projeto coloca o Brasil entre os líderes em aviação sustentável. A FAB pretende substituir progressivamente seus modelos mais antigos. Especialistas afirmam que o setor militar impulsionará tecnologias verdes nos próximos anos.', {'entities': [(2, 9, 'ORG'), (38, 60, 'ORG'), (311, 318, 'ORG'), (320, 340, 'PER'), (374, 380, 'LOC'), (288, 293, 'LOC'), (297, 307, 'CARGO'), (123, 142, 'TEMA'), (401, 420, 'TEMA'), (547, 565, 'TEMA')]}), ('A cidade de Recife implantou o primeiro corredor de ônibus 100% autônomo do país. Os veículos utilizam sensores, GPS e inteligência artificial para circular sem motorista em horários específicos. O projeto foi desenvolvido em parceria com a Universidade Federal de Pernambuco. A prefeitura afirma que o sistema reduzirá atrasos e ampliará a segurança. Motoristas do transporte coletivo criticaram a falta de diálogo sobre o impacto trabalhista. Testes seguirão por seis meses antes da expansão para outras linhas.', {'entities': [(12, 18, 'LOC'), (59, 63, 'VALOR'), (241, 275, 'ORG'), (103, 111, 'TEMA'), (119, 142, 'TEMA')]}), ('O Instituto Brasileiro de Geografia e Estatística (IBGE) divulgou que a população brasileira chegou a 205,7 milhões de habitantes em 2025. O crescimento foi puxado principalmente pelas regiões Norte e Nordeste. A taxa de natalidade continua em queda no Sudeste. O levantamento mostra também aumento da expectativa de vida, agora estimada em 77,6 anos. Economistas afirmam que o envelhecimento populacional exigirá maior investimento em saúde e previdência. O governo estuda políticas para incentivar a natalidade.', {'entities': [(2, 49, 'ORG'), (51, 55, 'ORG'), (102, 115, 'VALOR'), (193, 198, 'LOC'), (201, 209, 'LOC'), (253, 260, 'LOC'), (213, 231, 'TEMA'), (302, 321, 'TEMA'), (378, 405, 'TEMA'), (436, 441, 'TEMA'), (444, 455, 'TEMA')]}), ('O Ministério dos Transportes iniciou a construção de um túnel subfluvial que ligará Salvador a Itaparica. A obra, considerada a maior do tipo na América Latina, terá 12 km de extensão. A previsão de entrega é 2031. O projeto promete reduzir o tempo de deslocamento em 70%. Ambientalistas pedem mais estudos sobre o impacto marinho. O governo da Bahia afirma que a obra seguirá normas internacionais de proteção ambiental.', {'entities': [(2, 28, 'ORG'), (84, 92, 'LOC'), (95, 104, 'LOC'), (145, 159, 'LOC'), (345, 350, 'LOC'), (166, 171, 'VALOR'), (268, 271, 'VALOR'), (402, 420, 'TEMA')]}), ('A Bolsa de Valores de São Paulo (B3) anunciou a criação de um índice exclusivo para startups listadas. O novo indicador, chamado de B3 Tech Start, incluirá empresas de tecnologia com receita anual inferior a R$ 500 milhões. A iniciativa busca atrair investidores interessados em inovação. Analistas avaliam que o índice pode impulsionar o ecossistema de empreendedorismo no país. A B3 informou que as primeiras empresas devem estrear ainda este ano.', {'entities': [(168, 178, 'TEMA'), (208, 222, 'VALOR'), (354, 370, 'TEMA'), (22, 31, 'LOC')]}), ('A produção de café no Brasil deve atingir recorde histórico em 2025, segundo a Conab. A estimativa é de 77 milhões de sacas colhidas. O clima favorável e o uso crescente de tecnologia agrícola impulsionaram os resultados. Minas Gerais segue como maior produtor nacional. Exportadores comemoram e esperam expansão para novos mercados asiáticos. O preço do café pode cair no mercado interno devido à alta oferta.', {'entities': [(22, 28, 'LOC'), (222, 234, 'LOC'), (104, 114, 'VALOR'), (79, 84, 'ORG'), (173, 192, 'TEMA')]}), ('A prefeitura do Rio de Janeiro lançou um programa de revitalização da orla de Copacabana. O projeto inclui novas ciclovias, reformas de quiosques e ampliação das áreas verdes. As obras devem durar dois anos. Moradores apoiam a iniciativa, mas pedem mais atenção à segurança pública. O município espera atrair mais turistas internacionais com a modernização. O Ministério do Turismo prometeu liberar recursos adicionais.', {'entities': [(16, 30, 'LOC'), (78, 88, 'LOC'), (360, 381, 'ORG'), (264, 281, 'TEMA'), (344, 356, 'TEMA')]}), ('Pesquisadores da Universidade Federal do Paraná descobriram uma nova espécie de peixe em rios da Mata Atlântica. O animal, de apenas 3 cm, foi batizado de Characidium paranaensis. A espécie vive em águas extremamente limpas, o que indica bom estado de conservação da região. Os cientistas destacam a importância de proteger rios ameaçados pela poluição. O estudo será publicado na Biological Conservation.', {'entities': [(17, 47, 'ORG'), (381, 404, 'ORG'), (133, 137, 'VALOR'), (344, 352, 'TEMA')]}), ('O Ministério da Cultura anunciou uma nova política de incentivo para museus regionais. O programa prevê R$ 500 milhões para modernização de acervos e digitalização de documentos históricos. A ministra Margareth Menezes afirmou que o objetivo é democratizar o acesso ao patrimônio cultural. Museus do Norte e Nordeste serão prioridade. Especialistas comemoram o investimento inédito.', {'entities': [(2, 23, 'ORG'), (300, 305, 'LOC'), (308, 316, 'LOC'), (104, 118, 'VALOR'), (201, 218, 'PER'), (192, 200, 'CARGO'), (124, 136, 'TEMA'), (269, 288, 'TEMA')]}), ('O setor de energia eólica bateu recorde de geração no Nordeste nesta semana, suprindo 65% da demanda elétrica da região. Segundo o ONS, o vento forte favoreceu a produção em parques do Rio Grande do Norte e da Bahia. O governo estima que a fonte continuará crescendo nos próximos anos. Especialistas defendem expansão da rede de transmissão para evitar desperdício de energia gerada.', {'entities': [(54, 62, 'LOC'), (86, 89, 'VALOR'), (131, 134, 'ORG'), (185, 204, 'LOC'), (210, 215, 'LOC'), (368, 375, 'TEMA')]}), ('A Câmara dos Deputados aprovou a criação do Fundo Nacional de Segurança Digital. O novo órgão terá como missão combater crimes cibernéticos e proteger dados sensíveis do governo. A medida foi apoiada por todas as bancadas. O ministro da Justiça, Ricardo Lewandowski, destacou que ataques virtuais aumentaram 30% no último ano. O fundo financiará treinamento de agentes e novas tecnologias de proteção.', {'entities': [(2, 22, 'ORG'), (120, 139, 'TEMA'), (225, 244, 'CARGO'), (246, 265, 'PER'), (308, 311, 'VALOR'), (377, 388, 'TEMA')]}), ('O metrô de São Paulo iniciou testes com portas automáticas nas plataformas da Linha 1-Azul. O objetivo é reduzir acidentes e melhorar a fluidez do embarque. A implantação completa deve acontecer até 2027. Passageiros elogiaram a novidade. Técnicos alertam para a necessidade de adequações estruturais em algumas estações mais antigas.', {'entities': [(11, 20, 'LOC'), (113, 122, 'TEMA')]}), ('A cidade de Manaus inaugurou seu primeiro hospital veterinário público. A unidade oferece atendimento gratuito para animais domésticos e de rua. O prefeito afirmou que o objetivo é reduzir o abandono e promover saúde animal. A clínica conta com centro cirúrgico, laboratório e setor de vacinação. ONGs comemoraram a iniciativa e pedem expansão para outros municípios do Amazonas.', {'entities': [(12, 18, 'LOC'), (370, 378, 'LOC'), (297, 301, 'ORG'), (147, 155, 'CARGO'), (211, 223, 'TEMA'), (286, 295, 'TEMA')]}), ('O Ministério da Defesa confirmou a criação de uma força-tarefa para combater o tráfico de armas na fronteira com o Paraguai. O grupo integrará Exército, Polícia Federal e Receita Federal. A operação pretende reduzir a entrada ilegal de armamentos no Brasil. Especialistas em segurança pública elogiaram a ação, mas defendem cooperação internacional mais ampla.', {'entities': [(2, 22, 'ORG'), (250, 256, 'LOC'), (115, 123, 'LOC'), (79, 95, 'TEMA')]}), ('A Agência Nacional de Energia Elétrica autorizou a construção de uma nova usina solar no Piauí. O empreendimento terá capacidade para abastecer 800 mil residências. O investimento total é de R$ 3,2 bilhões. A empresa responsável promete gerar 5 mil empregos diretos durante a obra. A usina deve entrar em operação em 2028.', {'entities': [(2, 38, 'ORG'), (89, 94, 'LOC'), (144, 151, 'VALOR'), (191, 205, 'VALOR'), (243, 248, 'VALOR'), (74, 85, 'TEMA'), (98, 112, 'TEMA')]}), ('Pesquisadores da Fiocruz identificaram uma mutação inédita em mosquitos Aedes aegypti no litoral paulista. A alteração genética pode aumentar a resistência do mosquito a inseticidas. O Ministério da Saúde afirmou que monitorará o avanço da mutação. Especialistas pedem reforço das campanhas de combate ao vetor. A descoberta foi publicada na revista PLOS Neglected Tropical Diseases.', {'entities': [(17, 24, 'ORG'), (185, 204, 'ORG'), (72, 85, 'TEMA'), (109, 127, 'TEMA')]}), ('O Tribunal Superior Eleitoral lançou uma plataforma de verificação de fatos integrada às urnas eletrônicas. Eleitores poderão consultar informações sobre candidatos e checar notícias suspeitas. O ministro Alexandre de Moraes afirmou que a ferramenta é crucial para combater desinformação. A novidade estreia nas eleições municipais de 2026.', {'entities': [(2, 29, 'ORG'), (205, 224, 'PER'), (196, 204, 'CARGO'), (312, 320, 'TEMA')]}), ('A economia brasileira registrou crescimento de 2,3% no segundo trimestre de 2025, de acordo com o IBGE. O setor de serviços foi o principal responsável pela alta. A agropecuária apresentou leve retração devido à seca no Centro-Oeste. O Ministério da Fazenda comemorou o resultado e disse que o país mantém trajetória positiva. Analistas preveem desaceleração no final do ano.', {'entities': [(44, 51, 'VALOR'), (98, 102, 'ORG'), (165, 177, 'TEMA'), (220, 232, 'LOC'), (236, 257, 'ORG')]}), ('A Polícia Federal realizou nesta quinta-feira uma megaoperação contra fraudes em licitações de obras escolares. Foram cumpridos 42 mandados de busca e apreensão em seis estados. Segundo as investigações, empresas superfaturavam contratos e entregavam materiais de baixa qualidade. O Ministério da Educação afirmou que colaborará com todas as apurações. Políticos locais são suspeitos de envolvimento.', {'entities': [(2, 17, 'ORG'), (128, 130, 'VALOR'), (164, 168, 'VALOR'), (283, 305, 'ORG'), (70, 77, 'TEMA')]}), ('O Ministério da Saúde anunciou a ampliação do programa Farmácia Popular para incluir medicamentos para tratamento de doenças autoimunes, como lúpus e artrite reumatoide. A iniciativa deve beneficiar cerca de 3 milhões de pessoas. Segundo a ministra Nísia Trindade, o objetivo é reduzir o custo do tratamento e ampliar o acesso à saúde especializada. Farmácias conveniadas terão até 90 dias para se adaptar às novas regras.', {'entities': [(2, 21, 'ORG'), (208, 217, 'VALOR'), (249, 263, 'PER'), (240, 248, 'ORG'), (117, 135, 'TEMA'), (329, 348, 'TEMA')]}), ('A cidade de Curitiba iniciará em 2026 um projeto piloto de coleta de lixo totalmente automatizada. Os caminhões utilizarão braços robóticos para recolher resíduos sem intervenção humana. A prefeitura afirma que a medida reduzirá acidentes de trabalho e aumentará a eficiência. O projeto será testado em cinco bairros da capital paranaense.', {'entities': [(12, 20, 'LOC'), (303, 308, 'VALOR'), (229, 238, 'TEMA')]}), ('A Petrobras concluiu a instalação de uma nova plataforma de exploração de petróleo no pré-sal da Bacia de Santos. A unidade, chamada P-85, tem capacidade para produzir 250 mil barris por dia. A estatal afirmou que a plataforma utiliza tecnologia de última geração para reduzir emissões poluentes. O governo espera aumento nas exportações de petróleo no próximo semestre.', {'entities': [(2, 11, 'ORG'), (168, 175, 'VALOR'), (235, 245, 'TEMA'), (97, 112, 'LOC'), (60, 82, 'TEMA')]}), ('O governo do Ceará lançou o programa Cidade Conectada, que promete levar internet de fibra óptica gratuitamente a 200 mil famílias de baixa renda. O projeto será executado em parceria público-privada. O governador afirmou que a inclusão digital é essencial para educação e empregabilidade. Escolas estaduais serão priorizadas na primeira fase.', {'entities': [(13, 18, 'LOC'), (73, 81, 'TEMA'), (114, 121, 'VALOR'), (228, 244, 'TEMA'), (262, 270, 'TEMA'), (273, 288, 'TEMA')]}), ('A Câmara Municipal de Porto Alegre aprovou um plano de emergência para enchentes. O projeto inclui novas estações de bombeamento e ampliação do sistema de drenagem urbana. Após as fortes chuvas registradas no ano anterior, moradores pressionaram por ações mais rápidas. As obras devem começar no próximo trimestre.', {'entities': [(2, 18, 'ORG'), (22, 34, 'LOC'), (71, 80, 'TEMA')]}), ('A Marinha do Brasil lançou ao mar seu terceiro submarino da classe Riachuelo, o Tonelero. A cerimônia ocorreu no estaleiro de Itaguaí, no Rio de Janeiro. A embarcação conta com sistemas avançados de navegação e armamento. O presidente Lula destacou que o projeto fortalece a indústria naval brasileira. O submarino entrará em operação em 2027.', {'entities': [(126, 133, 'LOC'), (138, 152, 'LOC'), (235, 239, 'PER'), (224, 234, 'CARGO')]}), ('A Agência Nacional de Aviação Civil autorizou voos comerciais com aeronaves elétricas de pequeno porte no Brasil. O primeiro modelo aprovado, fabricado por uma startup paulista, poderá transportar até oito passageiros. O governo afirma que a medida reduzirá emissões e custos operacionais. As primeiras rotas serão regionais.', {'entities': [(2, 35, 'ORG'), (66, 85, 'TEMA'), (106, 112, 'LOC'), (201, 205, 'VALOR'), (269, 288, 'TEMA')]}), ('A Universidade Federal da Bahia criou um laboratório especializado em estudos sobre racismo estrutural. O projeto recebeu investimentos de R$ 12 milhões do governo federal. Pesquisadores irão desenvolver políticas públicas e ferramentas educacionais. Líderes comunitários comemoraram a iniciativa.', {'entities': [(2, 31, 'ORG'), (84, 102, 'TEMA'), (139, 152, 'VALOR')]}), ('O setor automotivo registrou crescimento de 9% nas vendas de carros híbridos em 2025. Segundo a Anfavea, o aumento é resultado de incentivos fiscais e da expansão de pontos de recarga. Modelos nacionais estão ganhando espaço no mercado. Especialistas acreditam que os híbridos devem superar os veículos a combustão até 2030.', {'entities': [(44, 46, 'VALOR'), (61, 76, 'TEMA'), (96, 103, 'ORG'), (130, 148, 'TEMA')]}), ('A prefeitura de Belém anunciou a criação de parques urbanos flutuantes na Baía do Guajará. As estruturas serão construídas com materiais biodegradáveis e servirão como áreas de lazer e de preservação ambiental. O projeto faz parte da preparação da cidade para sediar a COP 30. Moradores elogiaram a proposta inovadora.', {'entities': [(16, 21, 'LOC'), (82, 89, 'LOC'), (269, 275, 'ORG'), (188, 209, 'TEMA'), (127, 151, 'TEMA')]}), ('O Ministério da Justiça firmou parceria com empresas de tecnologia para desenvolver um sistema de reconhecimento facial capaz de identificar pessoas desaparecidas. O projeto-piloto será implantado em São Paulo e Brasília. ONGs afirmam que a ferramenta pode acelerar investigações, mas pedem garantias de privacidade.', {'entities': [(2, 23, 'ORG'), (200, 209, 'LOC'), (212, 220, 'LOC'), (222, 226, 'ORG'), (56, 66, 'TEMA'), (141, 162, 'TEMA')]}), ('A Secretaria de Educação do Amazonas anunciou a distribuição de tablets para alunos do ensino médio de escolas estaduais. Ao todo, 180 mil dispositivos serão entregues. O governo afirma que a medida reduz desigualdades no acesso a materiais pedagógicos. Professores receberão capacitação para uso da tecnologia em sala de aula.', {'entities': [(2, 36, 'ORG'), (131, 138, 'VALOR'), (254, 265, 'CARGO'), (300, 310, 'TEMA'), (205, 218, 'TEMA')]}), ('A economia do Nordeste deve crescer 3,1% em 2025, segundo estimativas da Sudene. O setor de energias renováveis e o turismo são os principais motores da expansão. Estados como Rio Grande do Norte e Ceará puxam o avanço. Empresários estão otimistas com novos investimentos internacionais na região.', {'entities': [(14, 22, 'LOC'), (36, 40, 'VALOR'), (73, 79, 'ORG'), (176, 195, 'LOC'), (198, 203, 'LOC'), (92, 111, 'TEMA'), (116, 123, 'TEMA')]}), ('A Polícia Rodoviária Federal lançou um sistema com drones para monitorar rodovias em tempo real. Os equipamentos serão utilizados para identificar acidentes, furtos de carga e congestionamentos. A PRF afirma que a tecnologia permitirá respostas mais rápidas e maior segurança aos motoristas.', {'entities': [(2, 28, 'ORG'), (197, 200, 'ORG'), (214, 224, 'TEMA'), (147, 156, 'TEMA'), (266, 275, 'TEMA')]}), ('A cidade de Florianópolis promoveu a primeira Conferência Nacional de Cidades Inteligentes. O evento reuniu especialistas de 15 países para debater soluções tecnológicas para transporte, energia e sustentabilidade urbana. O encontro resultou na assinatura de um acordo de cooperação internacional.', {'entities': [(12, 25, 'LOC'), (125, 127, 'VALOR'), (148, 169, 'TEMA'), (187, 194, 'TEMA'), (197, 220, 'TEMA')]}), ('A Gol Linhas Aéreas anunciou a abertura de uma nova rota direta entre Manaus e Bogotá. A empresa busca fortalecer sua presença na América do Sul. O Ministério do Turismo afirma que a conexão deve ampliar o fluxo de turistas na região amazônica. A operação começará em março de 2026.', {'entities': [(2, 19, 'ORG'), (70, 76, 'LOC'), (79, 85, 'LOC'), (130, 144, 'LOC'), (148, 169, 'ORG'), (227, 243, 'LOC')]}), ('O Instituto Nacional de Pesquisas Espaciais lançou um satélite de monitoramento climático desenvolvido integralmente no Brasil. O equipamento fornecerá dados em tempo real sobre chuvas e queimadas. Cientistas afirmam que o satélite melhorará previsões e ações de prevenção. O projeto contou com apoio da Agência Espacial Europeia.', {'entities': [(2, 43, 'ORG'), (120, 126, 'LOC'), (304, 329, 'ORG'), (66, 89, 'TEMA')]}), ('A prefeitura de Fortaleza inaugurou o maior centro de reciclagem do Nordeste. A unidade tem capacidade para processar 400 toneladas de resíduos por dia. Catadores organizados em cooperativas participarão da gestão. O governo municipal afirma que a iniciativa reduzirá o lixo enviado a aterros sanitários.', {'entities': [(16, 25, 'LOC'), (68, 76, 'LOC'), (118, 131, 'VALOR'), (54, 64, 'TEMA'), (285, 303, 'TEMA')]}), ('O Banco Central anunciou a expansão do Real Digital para transações internacionais. A fase de testes incluirá parcerias com bancos do Chile e de Portugal. A instituição afirma que o objetivo é reduzir custos e agilizar transferências. Especialistas apontam que o Brasil está entre os países mais avançados em moeda digital pública.', {'entities': [(2, 15, 'ORG'), (134, 139, 'LOC'), (145, 153, 'LOC'), (263, 269, 'LOC'), (57, 82, 'TEMA')]}), ('Pesquisadores da Universidade Federal de Mato Grosso identificaram um fungo capaz de degradar plástico em até 45 dias. A descoberta pode revolucionar o manejo de resíduos sólidos. A equipe agora busca formas de produzir a enzima em larga escala. O estudo será enviado para publicação na revista Science.', {'entities': [(17, 52, 'ORG'), (110, 112, 'VALOR'), (287, 302, 'ORG'), (162, 178, 'TEMA')]}), ('O Ministro da Fazenda, Fernando Haddad, detalhou em uma longa coletiva de imprensa em Brasília o novo arcabouço fiscal. A proposta, que visa substituir o antigo teto de gastos, foi apresentada a jornalistas e economistas. Analistas econômicos reagiram com cautela inicial, afirmando que, embora a intenção de controlar as contas públicas seja clara, a execução dependerá de negociação intensa. O foco agora se volta para o Congresso Nacional, onde o Presidente da Câmara, Arthur Lira, e o Presidente do Senado, Rodrigo Pacheco, terão papel crucial na aprovação e em possíveis alterações no texto.', {'entities': [(2, 21, 'CARGO'), (23, 38, 'PER'), (472, 483, 'PER'), (511, 526, 'PER'), (489, 509, 'CARGO'), (450, 470, 'CARGO'), (423, 441, 'ORG'), (86, 94, 'LOC'), (222, 242, 'CARGO'), (102, 118, 'TEMA'), (322, 337, 'TEMA')]}), ('A Ministra da Saúde, Nísia Trindade, anunciou oficialmente a expansão da campanha de vacinação contra a dengue. A nova fase incluirá adolescentes de 10 a 14 anos em todo o território nacional, priorizando áreas de alta transmissão. Pesquisadores da Fiocruz, que participaram do desenvolvimento e validação dos testes, celebraram a medida como um avanço na saúde pública. No entanto, a ministra fez um alerta, reforçando que o combate ao mosquito Aedes aegypti, vetor da doença, continua sendo a principal estratégia de prevenção e deve envolver toda a sociedade.', {'entities': [(2, 19, 'CARGO'), (385, 393, 'CARGO'), (21, 35, 'PER'), (104, 110, 'TEMA'), (149, 151, 'VALOR'), (154, 156, 'VALOR'), (249, 256, 'ORG'), (356, 369, 'TEMA'), (446, 459, 'TEMA')]}), ('A Ministra do Meio Ambiente e Mudança do Clima, Marina Silva, promoveu uma reunião de emergência com governadores dos estados que compõem a Amazônia Legal. O principal tema foi a reestruturação e aplicação dos recursos do Fundo Amazônia. Durante o encontro, foi decidido que o Ibama, órgão de fiscalização vinculado ao ministério, receberá fundos emergenciais para intensificar as operações de combate ao garimpo ilegal. Especialistas em conservação elogiaram a iniciativa, mas cobraram ações permanentes contra o crime ambiental.', {'entities': [(2, 27, 'CARGO'), (48, 60, 'PER'), (140, 148, 'LOC'), (222, 236, 'TEMA'), (277, 282, 'ORG'), (405, 419, 'TEMA'), (514, 529, 'TEMA')]}), ('O Presidente do Banco Central, Roberto Campos Neto, deu declarações que agitaram o mercado financeiro. Ele sinalizou que o Comitê de Política Monetária (Copom) deve manter uma postura vigilante e técnica contra a inflação, mesmo com a pressão por cortes mais rápidos na taxa Selic. Segundo Campos Neto, o cenário internacional ainda exige cautela. Após o discurso, economistas de grandes bancos revisaram suas projeções para a Selic no final do ano, prevendo agora um ritmo mais lento de queda nos juros básicos da economia.', {'entities': [(2, 29, 'CARGO'), (31, 50, 'PER'), (83, 101, 'TEMA'), (123, 151, 'ORG'), (153, 158, 'ORG'), (213, 221, 'TEMA'), (270, 280, 'TEMA'), (290, 301, 'PER'), (498, 503, 'TEMA'), (515, 523, 'TEMA')]}), ('O Ministro da Educação, Camilo Santana, lançou oficialmente o programa "Pé de Meia", uma nova política de poupança destinada a incentivar a permanência de jovens de baixa renda no ensino médio. O programa prevê um auxílio financeiro mensal e um bônus na conclusão dos estudos. Professores e diretores de escolas públicas, que participaram do evento de lançamento em Brasília, elogiaram a iniciativa. Eles destacam que a evasão escolar é um dos maiores desafios do país e que o incentivo financeiro pode ser decisivo para milhões de famílias.', {'entities': [(2, 22, 'CARGO'), (24, 38, 'PER'), (214, 232, 'TEMA'), (277, 288, 'CARGO'), (291, 300, 'CARGO'), (366, 374, 'LOC'), (420, 434, 'TEMA'), (477, 497, 'TEMA')]}), ('Em uma audiência pública no Senado, o Ministro da Justiça e Segurança Pública, Ricardo Lewandowski, defendeu enfaticamente a implementação de câmeras corporais em todas as polícias estaduais e federais. Ele argumentou que a medida é essencial para a transparência das ações policiais e para a proteção tanto do cidadão quanto do próprio agente. Especialistas em segurança pública presentes no debate apresentaram dados que mostram a redução da letalidade policial onde os equipamentos já são usados, embora sindicatos de policiais peçam regras mais claras sobre o uso das imagens.', {'entities': [(38, 77, 'CARGO'), (79, 98, 'PER'), (444, 463, 'TEMA'), (362, 379, 'TEMA')]}), ('O Ministro dos Transportes, Renan Filho, realizou uma inspeção técnica nas obras de duplicação da BR-101, no trecho que atravessa o Nordeste. A obra é considerada vital para o escoamento da produção regional. O ministro afirmou que o programa de concessões rodoviárias, que mistura investimento público e privado, é a principal estratégia do governo federal para destravar a logística nacional. Empresários do setor de transportes aguardam a conclusão da obra, que promete reduzir custos e o tempo de viagem significativamente.', {'entities': [(2, 26, 'CARGO'), (28, 39, 'PER'), (132, 140, 'LOC'), (211, 219, 'CARGO'), (246, 268, 'TEMA'), (282, 302, 'TEMA'), (375, 393, 'TEMA'), (419, 430, 'TEMA')]}), ('O Presidente da Confederação Nacional da Indústria (CNI), Ricardo Alban, apresentou formalmente ao governo federal um conjunto de propostas para a neoindustrialização do país. O plano, discutido com o Ministro do Desenvolvimento, Indústria e Comércio, Geraldo Alckmin, foca intensamente em inovação tecnológica e na transição energética. A CNI defende que, para aumentar a competitividade global, o Brasil precisa investir em indústrias verdes e na digitalização dos processos produtivos, exigindo uma política industrial clara de longo prazo.', {'entities': [(2, 50, 'CARGO'), (58, 71, 'PER'), (201, 228, 'CARGO'), (252, 267, 'PER'), (290, 310, 'TEMA'), (316, 336, 'TEMA'), (340, 343, 'ORG'), (52, 55, 'ORG'), (399, 405, 'LOC'), (426, 443, 'TEMA')]}), ('Uma equipe de pesquisadores da Unicamp (Universidade Estadual de Campinas) anunciou a descoberta de um novo biopesticida, desenvolvido a partir de uma bactéria encontrada no solo da Mata Atlântica. Segundo os cientistas, o produto tem alta eficácia contra pragas comuns da soja e do milho. A descoberta promete reduzir drasticamente o uso de agrotóxicos químicos na agricultura nacional, impactando positivamente a segurança alimentar e o meio ambiente. O próximo passo é conseguir investimento para a produção em larga escala.', {'entities': [(31, 38, 'ORG'), (40, 73, 'ORG'), (342, 362, 'TEMA'), (415, 434, 'TEMA'), (439, 452, 'TEMA'), (108, 120, 'TEMA')]}), ('A Ministra dos Povos Indígenas, Sonia Guajajara, usou a tribuna da ONU durante um fórum permanente para denunciar o avanço alarmante do desmatamento e do garimpo ilegal em terras Yanomami. A ministra apresentou relatórios que mostram o impacto devastador na saúde e na segurança dos povos originários. Em resposta, o governo federal, através de um comitê interministerial, prometeu novas ações emergenciais de saúde e o envio de mais agentes da Força Nacional para garantir a segurança e expulsar os invasores da região.', {'entities': [(2, 30, 'CARGO'), (32, 47, 'PER'), (67, 70, 'ORG'), (136, 148, 'TEMA'), (154, 161, 'TEMA'), (191, 199, 'CARGO'), (258, 263, 'TEMA'), (269, 278, 'TEMA'), (410, 415, 'TEMA'), (445, 459, 'ORG'), (476, 485, 'TEMA')]}), ('O IBGE divulgou hoje que o Produto Interno Bruto (PIB) do país teve um crescimento modesto no último trimestre, e analistas de mercado já preveem um cenário desafiador para o próximo ano. O setor de serviços continua sendo o principal motor da economia, impulsionado pelo consumo das famílias. Em contrapartida, a indústria de transformação enfrenta sérios desafios, sofrendo com os juros altos que encarecem o crédito e com a concorrência de produtos importados. Economistas pedem medidas de incentivo à produção industrial.', {'entities': [(2, 6, 'ORG'), (244, 252, 'TEMA'), (383, 388, 'TEMA'), (314, 323, 'TEMA')]}), ('Especialistas em direito digital e representantes de empresas de tecnologia debateram intensamente o projeto de lei sobre a regulamentação da inteligência artificial em uma audiência no Senado Federal. O texto busca equilibrar a necessidade de inovação tecnológica com a urgente proteção de dados dos cidadãos e a definição de responsabilidades sobre os direitos autorais. O relator do projeto, Senador Eduardo Gomes, afirmou que o desafio é criar uma legislação que não sufoque o desenvolvimento, mas que impeça o uso antiético da tecnologia.', {'entities': [(17, 32, 'TEMA'), (65, 75, 'TEMA'), (142, 165, 'TEMA'), (186, 200, 'ORG'), (244, 264, 'TEMA'), (403, 416, 'PER'), (395, 402, 'CARGO'), (481, 496, 'TEMA'), (532, 542, 'TEMA')]}), ('O Ministro da Agricultura e Pecuária, Carlos Fávaro, anunciou em evento em Brasília o novo Plano Safra, que contará com um volume recorde de financiamento para produtores rurais. Desta vez, o foco principal será em práticas sustentáveis. O governo informou que haverá linhas de crédito com juros reduzidos para quem investir em agricultura de baixo carbono, recuperação de pastagens e uso de bioinsumos, atendendo a uma demanda crescente do mercado internacional por produtos com rastreabilidade ambiental.', {'entities': [(2, 36, 'CARGO'), (38, 51, 'PER'), (75, 83, 'LOC'), (215, 236, 'TEMA'), (290, 295, 'TEMA'), (328, 339, 'TEMA'), (91, 102, 'TEMA')]}), ('A Presidenta da Petrobras, Magda Chambriard, reafirmou em uma reunião com investidores que a empresa manterá seus planos de investir pesado na exploração da Margem Equatorial. Ela classificou a área como a fronteira de petróleo mais promissora do país. A decisão, no entanto, gerou críticas imediatas de ambientalistas e uma nota técnica do Ministério do Meio Ambiente. A Ministra Marina Silva alertou publicamente sobre os riscos de um desastre ambiental em uma região de extrema sensibilidade ecológica.', {'entities': [(2, 25, 'CARGO'), (27, 43, 'PER'), (157, 174, 'TEMA'), (341, 368, 'ORG'), (372, 380, 'CARGO'), (381, 393, 'PER'), (437, 455, 'TEMA')]}), ('A Ministra da Cultura, Margareth Menezes, relançou oficialmente a Lei Rouanet, agora com novas diretrizes que visam democratizar o acesso aos recursos e descentralizar os investimentos. A principal mudança é a maior pontuação para projetos de regiões fora do eixo Rio-São Paulo. Produtores culturais de pequeno e médio porte, especialmente do Norte e Nordeste, celebraram a retomada dos incentivos, vendo nela uma oportunidade para financiar projetos que valorizem a cultura regional e que antes não conseguiam captação.', {'entities': [(2, 21, 'CARGO'), (23, 40, 'PER'), (343, 348, 'LOC'), (351, 359, 'LOC'), (467, 483, 'TEMA')]}), ('O Presidente do Supremo Tribunal Federal (STF), Ministro Luís Roberto Barroso, defendeu em um simpósio a continuidade da agenda de digitalização do judiciário brasileiro. O principal objetivo, segundo o ministro, é usar ferramentas de inteligência artificial para agilizar a triagem e a análise de processos repetitivos, que hoje congestionam os tribunais. A expectativa é que a tecnologia ajude a reduzir o gigantesco acervo processual e, consequentemente, entregue respostas mais rápidas à sociedade.', {'entities': [(2, 40, 'CARGO'), (42, 45, 'ORG'), (48, 56, 'CARGO'), (57, 77, 'PER'), (131, 144, 'TEMA'), (235, 258, 'TEMA'), (379, 389, 'TEMA')]}), ('O Prefeito de São Paulo, Ricardo Nunes, sancionou a controversa lei que revisa o Plano Diretor Estratégico da cidade. A aprovação ocorreu após meses de debates acalorados na Câmara Municipal. O tema mais polêmico foi o da verticalização, com a nova lei permitindo prédios mais altos em eixos de transporte. Vereadores da oposição e movimentos sociais criticaram a medida, alegando que ela beneficia a especulação imobiliária e irá adensar bairros que já sofrem com falta de infraestrutura, sem resolver o déficit habitacional.', {'entities': [(2, 10, 'CARGO'), (14, 23, 'LOC'), (25, 38, 'PER'), (222, 236, 'TEMA'), (295, 305, 'TEMA'), (401, 424, 'TEMA'), (474, 488, 'TEMA'), (505, 525, 'TEMA')]}), ('O Ministro do Trabalho e Emprego, Luiz Marinho, atuou pessoalmente como mediador na tensa negociação entre sindicatos de motoristas de aplicativo e as grandes plataformas digitais. A pauta principal dos trabalhadores é a regulamentação da profissão. Eles exigem um piso salarial mínimo por hora trabalhada e a inclusão obrigatória no sistema de contribuição previdenciária (INSS), garantindo direitos básicos como aposentadoria e auxílio-doença. As empresas, por sua vez, defendem um modelo mais flexível.', {'entities': [(2, 32, 'CARGO'), (34, 46, 'PER'), (221, 248, 'TEMA'), (374, 378, 'ORG'), (414, 427, 'TEMA'), (430, 444, 'TEMA')]}), ('O Presidente da Agência Espacial Brasileira (AEB), Marco Antonio Chamon, anunciou uma nova e ambiciosa parceria estratégica com a NASA, a agência espacial americana. O projeto visa incluir o Brasil de forma mais ativa no programa Artemis, que planeja levar humanos de volta à Lua. O foco da colaboração brasileira será no desenvolvimento de pequenos satélites (nanossatélites) para monitoramento lunar e em experimentos científicos em ambiente de microgravidade, abrindo campo para engenheiros e cientistas do país.', {'entities': [(2, 43, 'CARGO'), (45, 48, 'ORG'), (51, 71, 'PER'), (130, 134, 'ORG'), (191, 197, 'LOC'), (361, 375, 'TEMA')]}), ('Durante uma feira internacional de aviação na Europa, o CEO da Embraer, Francisco Gomes Neto, destacou o sucesso comercial sem precedentes do cargueiro militar KC-390 Millennium. A aeronave já foi vendida para várias forças aéreas da OTAN. O executivo afirmou que a divisão de defesa da companhia se tornou um pilar estratégico fundamental para o crescimento e a exportação, provando a capacidade da engenharia nacional em competir globalmente no setor de alta tecnologia e garantindo uma carteira de pedidos robusta para a próxima década.', {'entities': [(46, 52, 'LOC'), (56, 70, 'CARGO'), (72, 92, 'PER'), (234, 238, 'ORG'), (461, 471, 'TEMA')]})]

# Gera o gold
gold = convert_spacy_to_gold(dataset)

# Salva em JSON
with open("gold.json", "w", encoding="utf-8") as f:
    json.dump(gold, f, ensure_ascii=False, indent=4)


## Usando chave do gemini

In [10]:
load_dotenv()

KEY = os.getenv("GEMINI_API_KEY")

if not KEY:
    raise ValueError("ERRO: GEMINI_API_KEY não encontrada no arquivo .env")

# Configura a API
genai.configure(api_key=KEY)

model = genai.GenerativeModel("gemini-2.5-pro")

print("Gemini configurado com sucesso!")


Gemini configurado com sucesso!


## Função de avaliação

In [11]:
def evaluate_ner_by_label(gold, predictions):
   
    # Use defaultdict para criar automaticamente contadores para cada nova label
    stats_by_label = defaultdict(lambda: {"TP": 0, "FP": 0, "FN": 0})

    # Itera sobre cada par de (texto_gold, texto_predito)
    for gold_item, pred_item in zip(gold, predictions):
        
        # Extrai as entidades do gabarito 
        gold_entities = {
            (e["text"].strip(), e["label"])
            for e in gold_item["entities"]
        }

        # Extrai as entidades da predição
        if "predicted_entities" not in pred_item:
            pred_item["predicted_entities"] = []
            
        pred_entities = {
            (e["text"].strip(), e["label"])
            for e in pred_item["predicted_entities"]
        }

        # Calcula os TPs, FPs e FNs para ESTE texto
        TP_set = pred_entities & gold_entities  # O que acertou (está nos dois)
        FP_set = pred_entities - gold_entities  # O que previu, mas não estava no gabarito
        FN_set = gold_entities - pred_entities  # O que estava no gabarito, mas não previu

        # Adiciona os contadores para a label específica
        for (text, label) in TP_set:
            stats_by_label[label]["TP"] += 1
        
        for (text, label) in FP_set:
            stats_by_label[label]["FP"] += 1

        for (text, label) in FN_set:
            stats_by_label[label]["FN"] += 1

    
    results = {}
    overall_stats = {"TP": 0, "FP": 0, "FN": 0}

    sorted_labels = sorted(stats_by_label.keys())

    for label in sorted_labels:
        stats = stats_by_label[label]
        TP = stats["TP"]
        FP = stats["FP"]
        FN = stats["FN"]

        # Adiciona aos contadores gerais
        overall_stats["TP"] += TP
        overall_stats["FP"] += FP
        overall_stats["FN"] += FN

        # Calcula as métricas para esta label
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        results[label] = {
            "precision": round(precision, 4),
            "recall": round(recall, 4),
            "f1": round(f1, 4),
            "TP": TP,
            "FP": FP,
            "FN": FN
        }

    # Calcula as métricas gerais (Micro-average)
    TP = overall_stats["TP"]
    FP = overall_stats["FP"]
    FN = overall_stats["FN"]
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    results["overall"] = {
        "precision": round(precision, 4),
        "recall": round(recall, 4),
        "f1": round(f1, 4),
        "TP": TP,
        "FP": FP,
        "FN": FN
    }

    return results

In [12]:
def get_batches(data, batch_size):
    for i in range(0, len(data), batch_size):
        # Retorna uma "fatia" da lista original
        yield data[i:i + batch_size]

## Criação do prompt

In [13]:
def build_prompt(texts):
    import json

    # Prepara os textos em JSON válido dentro do prompt
    json_texts = json.dumps(texts, ensure_ascii=False, indent=2)

    return f"""
Você deve realizar Reconhecimento de Entidades Nomeadas (NER) nos textos fornecidos e extrair entidades pertencentes às seguintes classes:

PER — Pessoas (nomes próprios de indivíduos)
Exemplos: "Luiz Inácio Lula da Silva", "Fernando Haddad", "Elon Musk"

ORG — Organizações (empresas, instituições, governos, partidos)
Exemplos: "Petrobras", "Google", "Governo Federal", "BNDES"

LOC — Localizações (países, cidades, regiões, estados, rodovias, rios)
Exemplos: "Brasília", "Goiás", "BR-153", "São Paulo"

TEMA — Temas principais ou assuntos centrais do texto
Exemplos: "economia", "infraestrutura", "educação", "política", "saúde"

CARGO — Ocupações, funções e posições formais
Exemplos: "presidente", "ministro da Fazenda", "prefeito", "CEO"

VALOR — Expressões numéricas ou monetárias (inclui %, números, quantias financeiras)
Exemplos: "R$ 200 bilhões", "10%", "1,5 milhão", "2026", "R$ 3,50", "153"

Resposta OBRIGATORIAMENTE em JSON puro, SEM explicações, SEM comentários, SEM texto antes ou depois.

Formato obrigatório da resposta:
{{
  "results": [
    {{
      "text": "TEXTO",
      "predicted_entities": [
        {{"label": "PER", "text": "trecho exato"}},
        {{"label": "ORG", "text": "trecho exato"}}
      ]
    }}
  ]
}}

Regras obrigatórias:
- Preserve exatamente o texto original de cada entidade.
- Não altere, não normalize, não corrija o texto.
- Não retorne offsets.
- Se não houver entidades, use: "predicted_entities": []
- A resposta deve ser um JSON válido.
- O JSON deve corresponder exatamente à lista de textos enviada, na mesma ordem.
- Cada texto deve gerar exatamente um objeto dentro da lista "results".
- Não invente entidades que não existam no texto.

TEXTOS PARA PROCESSAR:
{json_texts}
"""


In [14]:
def get_predictions(text_list):
    prompt = build_prompt(text_list)

    response = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            response_mime_type="application/json"
        )
    )

    raw = response.text

    # Verifica se realmente veio JSON
    try:
        data = json.loads(raw)
    except Exception:
        print("ERRO: Gemini não retornou JSON válido.")
        print("==== RAW RESPONSE RECEBIDO ====")
        print(raw)
        raise

    return data["results"]


In [15]:
print("Extraindo entidades com Gemini...")

with open("gold.json", "r", encoding="utf-8") as f:
    gold_data = json.load(f)

texts = [item["text"] for item in gold_data]

all_predictions = []
BATCH_SIZE = 5 

total_batches = (len(texts) // BATCH_SIZE) + (1 if len(texts) % BATCH_SIZE > 0 else 0)
print(f"Iniciando processamento de {len(texts)} textos em {total_batches} lotes (tamanho={BATCH_SIZE})...")

# Itera sobre os lotes
for i, batch_texts in enumerate(get_batches(texts, BATCH_SIZE)):
    print(f"  Processando lote {i+1}/{total_batches}...")
    try:
        batch_predictions = get_predictions(batch_texts)
        
        all_predictions.extend(batch_predictions)
        
        print("    ...lote concluído. Pausando 1 segundo.")
        time.sleep(1) 

    except Exception as e:
        print(f"ERRO NO LOTE {i+1}: {e}")
        print("O processamento será interrompido.")
        break 

if len(all_predictions) == len(gold_data):
    with open("predictions.json", "w", encoding="utf-8") as f:
        json.dump(all_predictions, f, indent=2, ensure_ascii=False)
    print("\nPredictions salvas com sucesso!")

    predictions = all_predictions
    
else:
    print(f"\nERRO: O processamento foi interrompido ou falhou.")
    print(f"Esperado: {len(gold_data)} resultados. Obtido: {len(all_predictions)}")

Extraindo entidades com Gemini...
Iniciando processamento de 69 textos em 14 lotes (tamanho=5)...
  Processando lote 1/14...


    ...lote concluído. Pausando 1 segundo.
  Processando lote 2/14...
    ...lote concluído. Pausando 1 segundo.
  Processando lote 3/14...
    ...lote concluído. Pausando 1 segundo.
  Processando lote 4/14...
    ...lote concluído. Pausando 1 segundo.
  Processando lote 5/14...
    ...lote concluído. Pausando 1 segundo.
  Processando lote 6/14...
    ...lote concluído. Pausando 1 segundo.
  Processando lote 7/14...
    ...lote concluído. Pausando 1 segundo.
  Processando lote 8/14...
    ...lote concluído. Pausando 1 segundo.
  Processando lote 9/14...
    ...lote concluído. Pausando 1 segundo.
  Processando lote 10/14...
    ...lote concluído. Pausando 1 segundo.
  Processando lote 11/14...
    ...lote concluído. Pausando 1 segundo.
  Processando lote 12/14...
    ...lote concluído. Pausando 1 segundo.
  Processando lote 13/14...
    ...lote concluído. Pausando 1 segundo.
  Processando lote 14/14...
    ...lote concluído. Pausando 1 segundo.

Predictions salvas com sucesso!


## Avaliação

In [16]:
results = evaluate_ner_by_label(gold_data, predictions)
results

{'CARGO': {'precision': 0.3223,
  'recall': 0.78,
  'f1': 0.4561,
  'TP': 39,
  'FP': 82,
  'FN': 11},
 'LOC': {'precision': 0.77,
  'recall': 0.939,
  'f1': 0.8462,
  'TP': 77,
  'FP': 23,
  'FN': 5},
 'ORG': {'precision': 0.58,
  'recall': 0.87,
  'f1': 0.696,
  'TP': 87,
  'FP': 63,
  'FN': 13},
 'PER': {'precision': 1.0,
  'recall': 1.0,
  'f1': 1.0,
  'TP': 40,
  'FP': 0,
  'FN': 0},
 'TEMA': {'precision': 0.3475,
  'recall': 0.5325,
  'f1': 0.4206,
  'TP': 90,
  'FP': 169,
  'FN': 79},
 'VALOR': {'precision': 0.5385,
  'recall': 0.913,
  'f1': 0.6774,
  'TP': 42,
  'FP': 36,
  'FN': 4},
 'overall': {'precision': 0.5013,
  'recall': 0.77,
  'f1': 0.6073,
  'TP': 375,
  'FP': 373,
  'FN': 112}}